In [64]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [65]:
import pandas as pd
train_full = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv") #importing dataset and answers
test_full = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')

In [66]:
train_full.head() 

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [67]:
train_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [68]:
X_train_full = train_full.drop('Transported' , axis = 1) 
y_train_full = train_full['Transported'].copy()

In [69]:
from sklearn.model_selection import train_test_split 
random_state = 28
X_train, X_test, y_train,  y_test = train_test_split(X_train_full, y_train_full, test_size = 0.2, random_state = random_state, stratify = y_train_full)

In [70]:
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = random_state, stratify = y_test)

In [71]:
X_train_num = X_train.drop(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Name'], axis = 1)
X_val_num = X_val.drop(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Name'], axis = 1)
X_test_num = X_test.drop(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Name'], axis = 1)

In [72]:
X_train_num.head()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
7308,24.0,0.0,0.0,809.0,0.0,0.0
7401,25.0,0.0,0.0,0.0,0.0,0.0
4677,24.0,530.0,0.0,0.0,0.0,179.0
8459,27.0,0.0,984.0,0.0,13995.0,312.0
5643,19.0,781.0,0.0,295.0,613.0,0.0


In [73]:
from sklearn.pipeline import Pipeline #Pipeline for Numerical features 
from sklearn.impute import SimpleImputer #Instantiate the Imputer 
from sklearn.preprocessing import StandardScaler #Instantiate Standard Scalar
num_pipeline = Pipeline([
    ('imputer' , SimpleImputer(strategy='mean')) , 
    ('std_scaler' , StandardScaler()) , 
])

In [74]:
from sklearn.preprocessing import OneHotEncoder #Instatitate OneHot
cat_encoder = OneHotEncoder()

In [75]:
from sklearn.compose import ColumnTransformer #Full pipeline for both Numerical and Categorical features
X_train_num_attribs = list(X_train_num) 
X_train_cat_attribs = ['HomePlanet', 'CryoSleep', 'Destination', 'VIP']
full_pipeline = ColumnTransformer([
    ('num' , num_pipeline , X_train_num_attribs) , 
    ('cat' , OneHotEncoder() , X_train_cat_attribs) ,
])
full_pipeline.fit(X_train)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('std_scaler',
                                                  StandardScaler())]),
                                 ['Age', 'RoomService', 'FoodCourt',
                                  'ShoppingMall', 'Spa', 'VRDeck']),
                                ('cat', OneHotEncoder(),
                                 ['HomePlanet', 'CryoSleep', 'Destination',
                                  'VIP'])])

In [76]:
X_train_prepared = full_pipeline.transform(X_train)
X_val_prepared = full_pipeline.transform(X_val)
X_test_prepared = full_pipeline.transform(X_test)

In [77]:
X_train_prepared.shape

(6954, 20)

In [78]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 50)                1050      
                                                                 
 dense_6 (Dense)             (None, 50)                2550      
                                                                 
 alpha_dropout_2 (AlphaDropo  (None, 50)               0         
 ut)                                                             
                                                                 
 dense_7 (Dense)             (None, 50)                2550      
                                                                 
 alpha_dropout_3 (AlphaDropo  (None, 50)               0         
 ut)                                                             
                                                                 
 dense_8 (Dense)             (None, 1)                

In [79]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[20]):
 model = keras.models.Sequential()
 model.add(keras.layers.InputLayer(input_shape=input_shape))
 for layer in range(n_hidden):
     model.add(keras.layers.Dense(n_neurons, activation="selu",kernel_initializer='lecun_normal'))
 model.add(keras.layers.Dense(1, activation='sigmoid'))
 optimizer = keras.optimizers.RMSprop(lr=learning_rate)
 model.compile(loss="binary_crossentropy", optimizer=optimizer,metrics=["accuracy"])
 return model

In [80]:
keras_clf = keras.wrappers.scikit_learn.KerasClassifier(build_model)

/tmp/ipykernel_33/1341954221.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_clf = keras.wrappers.scikit_learn.KerasClassifier(build_model)


In [81]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
param_distribs = {
 "n_hidden": [1, 2, 3, 4, 5],
 "n_neurons": [50, 100, 200, 300, 400, 500],
 "learning_rate": reciprocal(3e-4, 3e-2),
}
rnd_search_cv = RandomizedSearchCV(keras_clf, param_distribs, n_iter=10, cv=3)
rnd_search_cv.fit(X_train_prepared, y_train, epochs=30, validation_data=(X_val_prepared, y_val),callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Epoch 1/30
145/145 [==============================] - 1s 5ms/step - loss: 0.5782 - accuracy: 0.7366 - val_loss: 0.4861 - val_accuracy: 0.7491
Epoch 2/30
145/145 [==============================] - 1s 4ms/step - loss: 0.4954 - accuracy: 0.7666 - val_loss: 0.4406 - val_accuracy: 0.7791
Epoch 3/30
145/145 [==============================] - 1s 4ms/step - loss: 0.4801 - accuracy: 0.7698 - val_loss: 0.4582 - val_accuracy: 0.7618
Epoch 4/30
145/145 [==============================] - 1s 4ms/step - loss: 0.4641 - accuracy: 0.7744 - val_loss: 0.4388 - val_accuracy: 0.7652
Epoch 5/30
145/145 [==============================] - 1s 4ms/step - loss: 0.4579 - accuracy: 0.7841 - val_loss: 0.4531 - val_accuracy: 0.7756
Epoch 6/30
145/145 [==============================] - 1s 4ms/step - loss: 0.4550 - accuracy: 0.7770 - val_loss: 0.4620 - val_accuracy: 0.7791
Epoch 7/30
145/145 [==============================] - 1s 4ms/step - loss: 0.4503 - accuracy: 0.7839 - val_loss: 0.4430 - val_accuracy: 0.7917
Epoch 

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7ad692ffa170>,
                   param_distributions={'learning_rate': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7ad692ff9b70>,
                                        'n_hidden': [1, 2, 3, 4, 5],
                                        'n_neurons': [50, 100, 200, 300, 400,
                                                      500]})

In [82]:
rnd_search_cv.best_params_


{'learning_rate': 0.009745023607424299, 'n_hidden': 3, 'n_neurons': 500}

In [83]:
rnd_search_cv.best_score_


0.7922059297561646

In [84]:
from functools import partial 

Dense = partial(keras.layers.Dense, 
               activation = 'selu',
               kernel_initializer='lecun_normal')

In [94]:
model = keras.models.Sequential([
    keras.layers.InputLayer(input_shape=20),
    Dense(500),
    Dense(500),
    keras.layers.AlphaDropout(rate=0.2),
    Dense(500),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(1, activation="sigmoid")
])

In [95]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=3)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_keras_model.h5", save_best_only=True)

In [96]:
model.summary()

Model: "sequential_34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_125 (Dense)           (None, 500)               10500     
                                                                 
 dense_126 (Dense)           (None, 500)               250500    
                                                                 
 alpha_dropout_6 (AlphaDropo  (None, 500)              0         
 ut)                                                             
                                                                 
 dense_127 (Dense)           (None, 500)               250500    
                                                                 
 alpha_dropout_7 (AlphaDropo  (None, 500)              0         
 ut)                                                             
                                                                 
 dense_128 (Dense)           (None, 1)               

In [97]:
model.compile(loss="binary_crossentropy",
 optimizer='RMSprop',
 metrics=["accuracy"])

In [98]:
 history = model.fit(X_train_prepared, y_train, epochs=50, validation_data=(X_val_prepared, y_val), callbacks=[lr_scheduler,early_stopping,checkpoint_cb])

Epoch 1/50
218/218 [==============================] - 2s 7ms/step - loss: 0.6094 - accuracy: 0.7384 - val_loss: 0.4969 - val_accuracy: 0.7802 - lr: 0.0010
Epoch 2/50
218/218 [==============================] - 1s 7ms/step - loss: 0.4844 - accuracy: 0.7601 - val_loss: 0.7780 - val_accuracy: 0.6835 - lr: 0.0010
Epoch 3/50
218/218 [==============================] - 1s 6ms/step - loss: 0.4589 - accuracy: 0.7793 - val_loss: 0.5746 - val_accuracy: 0.7779 - lr: 0.0010
Epoch 4/50
218/218 [==============================] - 1s 6ms/step - loss: 0.4498 - accuracy: 0.7837 - val_loss: 0.4922 - val_accuracy: 0.7860 - lr: 0.0010
Epoch 5/50
218/218 [==============================] - 1s 6ms/step - loss: 0.4492 - accuracy: 0.7794 - val_loss: 0.5976 - val_accuracy: 0.7664 - lr: 0.0010
Epoch 6/50
218/218 [==============================] - 1s 6ms/step - loss: 0.4442 - accuracy: 0.7880 - val_loss: 0.5520 - val_accuracy: 0.7445 - lr: 0.0010
Epoch 7/50
218/218 [==============================] - 1s 7ms/step - lo

In [99]:
X_test_prepared = full_pipeline.transform(test_full)

In [100]:
y_test = model.predict(X_test_prepared)

134/134 [==============================] - 0s 2ms/step


In [101]:
Transported = []
for proba_of_transportation in y_test:
    if proba_of_transportation > 0.5:
        Transported.append(True)
    else:
        Transported.append(False)

In [102]:
output = pd.DataFrame({'PassengerId': test_full.PassengerId, 'Transported': Transported})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
